In [ ]:
import numpy as np
import matplotlib.cbook as cbook
import matplotlib.image as image
import matplotlib.pyplot as plt
import cv2
import os
from yu_plot_axes import run_animation

def scale_image(filename_with_path, scaling_factor=1, mirror=False):
    with cbook.get_sample_data(filename_with_path) as file:
        original_image = image.imread(file)
    new_shape = (int(original_image.shape[1] / scaling_factor), 
                 int(original_image.shape[0] / scaling_factor))
    resized_image = cv2.resize(original_image, dsize=new_shape)
    if mirror:
        resized_image = resized_image[:,::-1,:]
    return resized_image

def prepare_image(filename_with_path, mirror=False):
    if filename_with_path.startswith('~'):
        filename_with_path = os.path.expanduser(filename_with_path)
    with cbook.get_sample_data(filename_with_path) as file:
        prepared_image = image.imread(file)
    if mirror:
        prepared_image = prepared_image[:,::-1,:]
    return prepared_image

def show_image(ax, prepared_image, origin, scaling_factor=1, where_position=None):
    extent=[origin[0], origin[0] + prepared_image.shape[0]*scaling_factor, 
            origin[1], origin[1] + prepared_image.shape[1]*scaling_factor]
    if 'R' in where_position:
        extent[0] -= prepared_image.shape[0]*scaling_factor
        extent[1] -= prepared_image.shape[0]*scaling_factor
    ax.imshow(prepared_image, extent=extent)        

def val_or_func_to_val(val_or_func, arg):
    if isinstance(val_or_func, int) or isinstance(val_or_func, float):
        val = val_or_func
    else:
        if isinstance(val_or_func, tuple):
            val = val_or_func[0] * arg + val_or_func[1]
        else:    
            val = val_or_func(arg)
    return val

def animate_images(i, ax, nb_frames, images_xs_ys, title_func):
    ax.clear()
    for spine_id in ['right', 'top', 'left']:
        ax.spines[spine_id].set_visible(False)
    ax.set_yticks([], [])
    ax.set_xlim(-50, 570)
    ax.set_ylim(0, 80)

    positions = []
    for img, x, y, scaling, where_position in images_xs_ys_scalings:
        x_value = val_or_func_to_val(val_or_func=x, arg=i)
        y_value = val_or_func_to_val(val_or_func=y, arg=i)
        positions.append(x_value)
        show_image(ax, img, (x_value, y_value), scaling, where_position)
    
    plot_title = title_func(positions)
    ax.set_title(plot_title)


In [ ]:
car_blue = prepare_image('~/Documents/Python/Tutoring/car_blue.png')
car_red = prepare_image('~/Documents/Python/Tutoring/car_red.png')

def show_distance(xs):
    return ("The distance between the cars is " + str(abs(xs[1] - xs[0]))
           + ". \nRed car is at " + str(xs[1]))

images_xs_ys_scalings = [[car_blue, (-5, 500), 10, 1., "L"],
                         [car_red,  (15, 0),  -20, 1., "R"]]

anim = run_animation(figsize=(15, 5), nb_frames=100, 
                     func_name=animate_images, fargs=[images_xs_ys_scalings, show_distance], interval=200)
anim